In [54]:
import numpy as np
import pandas as pd
import calendar

In [55]:
df = pd.read_excel("../data/denormalized-data.xlsx")
df.sample(5)

,Row ID,Order ID,Order Date,Delivery Date,Delivery Mode,Customer ID,Customer Name,Customer Segment,City,Country,Region,Product ID,Product Category,Sub-Category,Product Name,Unit cost,Quantity,Selling price,Discount
301,303,OrdID-2018-0003031,2018-12-11,2018-12-13,Pick up,CustID- 397,Godred Gyimah,Corporate,Ashaiman,Ghana,Greater Accra,ProdID-28001161,Electronics,Television,Samsung UN32J4001 32-Inch J4001-Series 720p HD...,3940,1,5674,0.000000
521,523,OrdID-2019-0005231,2019-08-13,2019-08-15,Pick up,CustID- 245,Tetteyfio Akuyoo,Corporate,Dzodze,Ghana,Volta,ProdID-28000761,Phone_Tablets,Telephones_Accessories,Panasonic Genuine HHR-4DPA/4B AAA NiMH Recharg...,380,17,483,0.140000
31,32,OrdID-2018-0000321,2018-01-26,2018-01-28,2-3 Day,CustID- 525,Peter Ankoma,Consumer,Axim,Ghana,Western,ProdID-28000321,Phone_Tablets,Laptop_Desktop accessories,Logitech C270 HD Computer Webcam Drive-Free wi...,638,5,850,0.002189
563,565,OrdID-2019-0005651,2019-09-24,2019-09-27,2-3 Day,CustID- 334,Elikem Kobla,Consumer,Effiduase,Ghana,Ashanti,ProdID-28000111,Electronics,Accessories_Supplies,TV One 1RK-5RU-PSU 5RU 250w Power supply and a...,591,16,798,0.082766
220,222,OrdID-2018-0002221,2018-09-04,2018-09-06,2-3 Day,CustID- 372,Antwi Frimpong,Corporate,Akatsi,Ghana,Volta,ProdID-28000801,Home_Office,Home_Kitchen,Plastic Storage Bowl Set - 17 Pieces Green,1007,8,1310,0.078913


In [56]:
df["Revenue"] = df["Quantity"] * df["Selling price"]
df["Revenue"].sum()

6998937

In [57]:
df["Cost"] = df["Quantity"] * df["Unit cost"]
df["Cost"].sum()

5270143

In [58]:
df["Profit"] = df["Revenue"] - df["Cost"]
df["Profit"].sum()

1728794

In [59]:
df["Profit Margin"] = df["Profit"]/df["Revenue"]
df["Profit Margin"].mean()

0.24729130544498248

In [60]:
months = df["Order Date"].dt.month
try:
    months = months.apply(lambda x: calendar.month_abbr[x])
except(TypeError):
    months = months
# trend_df = df.groupby([months]).agg(
    # {"Revenue":"sum", "Cost":"sum", "Profit":'sum', "Profit Margin":'mean'})
order = calendar.month_abbr
trend_df = df.groupby([months])[["Revenue", "Cost", "Profit"]].sum()
trend_df["Profit Margin"] = trend_df["Profit"]/trend_df["Revenue"]
trend_df = trend_df.reindex(order, axis=0)
trend_df.dropna(inplace=True)
trend_df = trend_df.reset_index()


trend_df

,Order Date,Revenue,Cost,Profit,Profit Margin
0,Jan,506145.0,376940.0,129205.0,0.255273
1,Feb,306517.0,230723.0,75794.0,0.247275
2,Mar,767026.0,560592.0,206434.0,0.269136
3,Apr,796906.0,597874.0,199032.0,0.249756
4,May,570425.0,437312.0,133113.0,0.233358
5,Jun,298017.0,233627.0,64390.0,0.216061
6,Jul,682283.0,508509.0,173774.0,0.254695
7,Aug,496796.0,377420.0,119376.0,0.240292
8,Sep,668480.0,518723.0,149757.0,0.224026
9,Oct,682572.0,511566.0,171006.0,0.250532


In [61]:
regions_df = df.groupby('Region')["Profit"].sum().sort_values(ascending=False)
regions_df = regions_df.reset_index()
regions_df["Region"] = regions_df["Region"].str.replace("Brong-Ahafo", "Brong Ahafo")
regions_df


,Region,Profit
0,Northern,400787
1,Ashanti,245647
2,Greater Accra,208897
3,Volta,184245
4,Western,183871
5,Central,112227
6,Upper West,111352
7,Upper East,104895
8,Eastern,91101
9,Brong Ahafo,85772


In [62]:
sub_category_df = df.groupby('Sub-Category')[["Revenue", "Cost", "Profit"]].sum().sort_values(by="Revenue",ascending=False)
sub_category_df["PM Tracker"] = sub_category_df["Profit"]/sub_category_df["Revenue"]
sub_category_df.drop("Profit",axis=1, inplace=True)
sub_category_df.loc['Grand Total'] = sub_category_df[["Revenue", "Cost"]].sum()
sub_category_df["PM Tracker"].loc['Grand Total'] = sub_category_df["PM Tracker"].mean()
sub_category_df = sub_category_df.reset_index()

#sub_category_df

In [63]:
sub_category_df["Revenue"] = sub_category_df["Revenue"].apply(lambda x: "{:,.1f}k".format((x/1000)))
sub_category_df["Cost"] = sub_category_df["Cost"].apply(lambda x: "{:,.1f}k".format((x/1000)))
#sub_category_df

In [64]:
sub_category_df["PM Tracker"] = sub_category_df["PM Tracker"].apply(lambda x: '🔴' if x<0.2
else '🟡' if x<0.25
else '🟢')

sub_category_df

,Sub-Category,Revenue,Cost,PM Tracker
0,Television,"2,001.3k","1,539.0k",🟡
1,Mobile Phones,"1,224.4k",913.6k,🟢
2,Telephones_Accessories,894.7k,679.3k,🟡
3,Home_Kitchen,887.2k,662.9k,🟢
4,Accessories_Supplies,471.8k,340.5k,🟢
5,Laptop_Desktop accessories,441.1k,328.1k,🟢
6,Wearable Technology,253.0k,191.6k,🟡
7,Home Audio,218.4k,159.7k,🟢
8,Furniture,182.9k,136.4k,🟢
9,Headphones,178.3k,131.8k,🟢


In [65]:
# def red_circle(v, props=""):
#     return props if v <0.2 else None


# sub_category_df[["PM Tracker"]] = sub_category_df[["PM Tracker"]].style.applymap(red_circle,
#  props="height: 25px; width: 25px; background-color: #c85533; border-radius: 50%;")


# sub_category_df

In [66]:

ghana_geo = pd.read_json("../data/ghana_regions.json")
ghana_geo = ghana_geo["features"]
#ghana_geo

In [67]:
import plotly.express as px
fig = px.choropleth(regions_df, geojson=ghana_geo, scope="africa", labels="Profit",locations="Region")
#fig.show()

In [68]:
#ghana_geo[0]

In [69]:
regions_geo = []
for region in ghana_geo:
    region_name = region["properties"]['NAME_1']
    geometry = region["geometry"]
    regions_geo.append({"type":"Feature", 
                        "geometry":geometry,
                        "id":region_name})
#regions_geo[0]

In [70]:
regions_geo_full = {"type":"Feature Collection", "features":regions_geo}

In [71]:
# fig = px.choropleth(regions_df, 
# geojson = regions_geo_full,
# color = regions_df["Profit"])

# fig.update_layout(
#     margin={'r':0,'t':0,'l':0,'b':0},
#     coloraxis_colorbar={
#         'title':'Profit per region',
#         'tickvals':regions_df["Profit"]     
#     }
# )
# fig.show()

In [72]:
def create_date_table(start='2018-01-01', end='2019-12-31'):
    date_df = pd.DataFrame({"Date": pd.date_range(start, end)})
    date_df["Day"] = date_df.Date.dt.day
    date_df["Month"] = date_df.Date.dt.month
    date_df["Month"] = date_df["Month"].apply(lambda x: calendar.month_abbr[x])
    date_df["Quarter"] = date_df.Date.dt.quarter
    date_df["Year"] = date_df.Date.dt.year
    return date_df

#create_date_table()

In [73]:
date_df = create_date_table()
date_df = date_df[date_df["Year"].isin([2019])]
date_df = date_df["Day"]
type(int(np.asarray(date_df.min())))

int

In [74]:
"".join(["Months"])

'Months'

In [75]:
df["Years"] = df["Delivery Date"].dt.year
df["Months"] = df["Delivery Date"].dt.month
df["Quarters"] = df["Delivery Date"].dt.quarter
df["Days"] = df["Delivery Date"].dt.day

period = "".join(["Months"])
df = df[df["Years"].isin([2018, 2019])]
df = df[df[period].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])]

profit = df["Profit"]
profit = "{:,}".format(profit.sum())
profit

'1,728,794'

In [76]:
dict_ = df.to_dict("series")
#dict_

In [77]:
df1 = pd.DataFrame(dict_)
#df1